In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/JeffSackmann/tennis_wta/master/wta_matches_2024.csv"
wta_2024 = pd.read_csv(url)
print(wta_2024.head())


  tourney_id tourney_name surface  draw_size tourney_level  tourney_date  \
0  2024-9900   United Cup    Hard         18             I      20240101   
1  2024-9900   United Cup    Hard         18             I      20240101   
2  2024-9900   United Cup    Hard         18             I      20240101   
3  2024-9900   United Cup    Hard         18             I      20240101   
4  2024-9900   United Cup    Hard         18             I      20240101   

   match_num  winner_id  winner_seed winner_entry  ... l_1stIn l_1stWon  \
0        299     216347          NaN          NaN  ...    27.0     13.0   
1        297     216347          NaN          NaN  ...    47.0     28.0   
2        295     201493          NaN          NaN  ...    82.0     47.0   
3        293     216347          NaN          NaN  ...    27.0     18.0   
4        291     201614          NaN          NaN  ...    72.0     51.0   

   l_2ndWon l_SvGms  l_bpSaved  l_bpFaced  winner_rank winner_rank_points  \
0       7.0    

In [2]:
years = range(2000, 2018)
base_url = "https://raw.githubusercontent.com/JeffSackmann/tennis_wta/master/wta_matches_{}.csv"

frames = []
for year in years:
    try:
        df = pd.read_csv(base_url.format(year))
        df["year"] = year
        frames.append(df)
    except Exception as e:
        print(f"Error loading {year}: {e}")

wta_all = pd.concat(frames, ignore_index=True)
print(wta_all.shape)


(51159, 50)


In [4]:
import os

# Create folder if it doesn't exist
os.makedirs("data/cleaned", exist_ok=True)

# Now save the CSV
wta_all.to_csv("data/cleaned/wta_all.csv", index=False)
print("✅ File saved successfully!")


✅ File saved successfully!
